In [13]:
## Keras layers for LSTM RNN 

from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, TimeDistributed, Flatten
from keras import regularizers, optimizers
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pywt


In [14]:
## load data
print('Loading data...')
X = np.genfromtxt('shuffledSet_w50_s10.txt')
X1 = np.reshape(X,(8430,50,11))

## ========================   headers  ====================================================
# Depth    ROP     WOB     RPM     Torque     Bit Size     GR     RHOB     NPHI     DT     RD
## ========================================================================================

print('data loaded')



Loading data...
data loaded


In [15]:
# split to test and train
X, Y = X1[:,:,0:6], X1[:,:,6:]

# split into train and test sets
x_train = X[:6400,:,[1, 2, 4]]
y_train = Y[:6400, :, 1]  # 0 for GR, 1 for RHOB

x_test = X[-1600:,:,[1, 2, 4]]
y_test = Y[-1600:, :, 1]


## sanity check
#x_train = X[0:1,:,:]
#y_train = Y[0:1, :, 1]  # 0 for GR, 1 for RHOB

#x_test = X[6744:,:,:]
#y_test = Y[6744:, :, 1]

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(6400, 50, 3) (6400, 50) (1600, 50, 3) (1600, 50)


The data set is uploaded and sliced as follows, [samples, timesteps, features] = $[m, T_x, x]$

Then we implement our BiLSTM model.

The model has the following architecture. ==>   

In [16]:
## Wavelet transform for later
#x = np.asarray(x_train[0,:,1])
#y = np.asarray(y_train[0,:])
#print(x.shape)
#cA, cD, cB, cC = np.asarray(pywt.wavedec(x, 'haar', mode='constant', level=3))
#aA, aD, aB, aC = np.asarray(pywt.wavedec(y, 'haar', mode='constant', level=3))

#plt.plot(aD)
#plt.plot(cD)              

In [17]:
def StatefulLSTM(x_train, y_train, batch_size, units, dropout, epochNo, learning_rate, x_test, y_test):
    
    # network
    input_dim = (x_train.shape[1],x_train.shape[2])
    print((input_dim))
    
    new_model = Sequential()
    model = Sequential()
    for i in range(len(units)-1):
        #print((units[i]))
        model.add(LSTM(units[i], batch_input_shape = (batch_size, input_dim[0], input_dim[1]), 
                       kernel_regularizer=regularizers.l2(0.01), 
                       recurrent_regularizer=regularizers.l2(0.01), 
                       bias_regularizer=regularizers.l2(0.01), 
                       activation='tanh', 
                        stateful=True, return_sequences=True))
        
        new_model.add(LSTM(units[i], batch_input_shape = (1, input_dim[0], input_dim[1]), 
                       kernel_regularizer=regularizers.l2(0.01), 
                       recurrent_regularizer=regularizers.l2(0.01), 
                       bias_regularizer=regularizers.l2(0.01), 
                       activation='tanh', 
                        stateful=True, return_sequences=True))
        
        new_model.add(Dropout(dropout))
        model.add(Dropout(dropout))



    model.add(LSTM(units[len(units)-1], batch_input_shape = (batch_size, input_dim[0], input_dim[1]),
                   kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                   bias_regularizer=regularizers.l2(0.01), 
                   activation='tanh', stateful=True))

    new_model.add(LSTM(units[len(units)-1], batch_input_shape = (1, input_dim[0], input_dim[1]),
                   kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                   bias_regularizer=regularizers.l2(0.01), 
                   activation='tanh', stateful=True))

    
    
    model.add(Dropout(dropout))
    model.add((Dense(input_dim[0])))

    
    new_model.add(Dropout(dropout))
    new_model.add((Dense(input_dim[0])))

    # try using different optimizers and different optimizer configs
    opt = optimizers.Adam(lr = learning_rate)
    model.compile(loss = 'mae', optimizer = opt)
    new_model.compile(loss = 'mae', optimizer = opt)

    
    for i in range(epochNo):
        print('epoch:',i+1)
        model.fit(x_train, y_train, epochs = 1, batch_size = batch_size, validation_data=(x_test, y_test), 
                  verbose=1, shuffle=False)
        model.reset_states()

    
    
    return model, new_model

In [18]:
def StatefulBiLSTM(x_train, y_train, batch_size, units, dropout, epochNo, learning_rate, x_test, y_test):
    
    # network
    input_dim = (x_train.shape[1],x_train.shape[2])
    print((input_dim))
    
    new_model = Sequential()
    model = Sequential()
    for i in range(len(units)-1):
        #print((units[i]))
        model.add(Bidirectional(LSTM(units[i], 
                       kernel_regularizer=regularizers.l2(0.01), 
                       recurrent_regularizer=regularizers.l2(0.01), 
                       bias_regularizer=regularizers.l2(0.01), 
                       activation='tanh', 
                        stateful=True, return_sequences=True), batch_input_shape = (batch_size, input_dim[0], input_dim[1])))
        
        new_model.add(Bidirectional(LSTM(units[i], 
                       kernel_regularizer=regularizers.l2(0.01), 
                       recurrent_regularizer=regularizers.l2(0.01), 
                       bias_regularizer=regularizers.l2(0.01), 
                       activation='tanh', 
                        stateful=True, return_sequences=True), batch_input_shape = (1, input_dim[0], input_dim[1])))
        
        new_model.add(Dropout(dropout))
        model.add(Dropout(dropout))



    model.add(Bidirectional(LSTM(units[len(units)-1],
                   kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                   bias_regularizer=regularizers.l2(0.01), 
                   activation='tanh', stateful=True), batch_input_shape = (batch_size, input_dim[0], input_dim[1])))

    new_model.add(Bidirectional(LSTM(units[len(units)-1], 
                   kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                   bias_regularizer=regularizers.l2(0.01), 
                   activation='tanh', stateful=True), batch_input_shape = (1, input_dim[0], input_dim[1])))

    
    
    model.add(Dropout(dropout))
    model.add((Dense(input_dim[0])))

    
    new_model.add(Dropout(dropout))
    new_model.add((Dense(input_dim[0])))

    # try using different optimizers and different optimizer configs
    opt = optimizers.Adam(lr = learning_rate)
    model.compile(loss = 'mae', optimizer = opt)
    new_model.compile(loss = 'mae', optimizer = opt)

    
    for i in range(epochNo):
        print('epoch:',i+1)
        model.fit(x_train, y_train, epochs = 1, batch_size = batch_size, validation_data=(x_test, y_test), 
                  verbose=1, shuffle=False)
        model.reset_states()

    
    
    return model, new_model

In [19]:
## My network
def StatelessLSTM(x_train):
    input_dim = (x_train.shape[1],x_train.shape[2])
    units = 25

    model = Sequential()
    #model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape = (x_train.shape[1],x_train.shape[2])))
    #model.add(Bidirectional(LSTM(20, return_sequences=True))
    model.add(LSTM(units, return_sequences=True, input_shape = (x_train.shape[1],x_train.shape[2]),
             kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                                bias_regularizer=regularizers.l2(0.01)))
    model.add(LSTM(units, return_sequences=True,kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                                bias_regularizer=regularizers.l2(0.01)))
    model.add(LSTM(units, return_sequences=True,kernel_regularizer=regularizers.l2(0.01), 
                   recurrent_regularizer=regularizers.l2(0.01), 
                                bias_regularizer=regularizers.l2(0.01)))
    model.add(LSTM(units))

    model.add((Dense(50)))

    # try using different optimizers and different optimizer configs
    model.compile(loss='mae', optimizer='adam')

    model.summary()
    return model

In [ ]:
units = [3, 3, 3]
dropout = 0.1
batch_size = 80  # a multiple of training set
epochNo = 400
learning_rate = 0.001
print(len(units))

model, new_model = StatefulBiLSTM(x_train, y_train, batch_size, units, dropout, epochNo, learning_rate, x_test, y_test)

#model, new_model = StatefulLSTM(x_train, y_train, batch_size, units, dropout, epochsNo, learning_rate, x_test, y_test)

# Prediction = model.fit(x_train, y_train, epochs=120, batch_size=192, validation_data=(x_test, y_test),\
#                        verbose=1, shuffle=False)


3
(50, 3)
epoch: 1
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 13s 2ms/step - loss: 1.4936 - val_loss: 1.3453
epoch: 2
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 988us/step - loss: 1.2590 - val_loss: 1.1388
epoch: 3
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 980us/step - loss: 1.0668 - val_loss: 0.9693
epoch: 4
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 982us/step - loss: 0.9300 - val_loss: 0.8696
epoch: 5
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 1ms/step - loss: 0.8481 - val_loss: 0.8084
epoch: 6
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 995us/step - loss: 0.7983 - val_loss: 0.7678
epoch: 7
Train on 6400 samples, v

6400/6400 [==============================] - 6s 952us/step - loss: 0.6514 - val_loss: 0.6406
epoch: 53
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 1ms/step - loss: 0.6507 - val_loss: 0.6403
epoch: 54
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 999us/step - loss: 0.6515 - val_loss: 0.6401
epoch: 55
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 1ms/step - loss: 0.6485 - val_loss: 0.6396
epoch: 56
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 994us/step - loss: 0.6501 - val_loss: 0.6395
epoch: 57
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [==============================] - 6s 952us/step - loss: 0.6499 - val_loss: 0.6388
epoch: 58
Train on 6400 samples, validate on 1600 samples
Epoch 1/1
6400/6400 [===========================

In [ ]:
## transfer the statefull LSTM to a proper dimension model 

opt = optimizers.Adam(lr = learning_rate)
old_weights = model.get_weights()
new_model.set_weights(old_weights)
# compile model
new_model.compile(loss = 'mae', optimizer = opt)



In [ ]:
# plot history
plt.plot(Prediction.history['loss'], label='train')
plt.plot(Prediction.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# make prediction
yhat = new_model.predict(x_test,1)

In [ ]:
sample = 900
plt.plot(X[sample, :, 0], y_test[sample,:])
plt.plot(X[sample, :, 0], yhat[sample,:])